# <b><span style='color:#2563eb'>00 | </span>Modele predykcji</b>

## <span style='color:#2563eb'>🔷 | <b></span>Import bibliotek</b>

In [1]:
# Set auto reload after making changes
%load_ext autoreload
%autoreload 2

import os

import numpy as np
import pandas as pd
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

# Modeling
import xgboost as xgb
import torch

from sklearn.model_selection import GridSearchCV

# Wrote myself
from source.CustomPlot import CustomPlot
from source.Utils import SplitDateColumn, AddPrefixToColumns, DescribeData
from sklearn.ensemble import RandomForestRegressor

# GPU or CPU use for model
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

## <span style='color:#2563eb'>🔷 | <b></span>Zbiory</b>

In [2]:
PATH = 'data/'

gas_prices = pd.read_csv(os.path.join(PATH, 'gas_prices.csv'),
                   dtype={'lowest_price_per_mwh': 'float64',
                          'highest_price_per_mwh': 'float64',
                          'data_block_id': 'int64'},
                   parse_dates=['forecast_date', 'origin_date'])

electricity_prices = pd.read_csv(os.path.join(PATH, 'electricity_prices.csv'),
                   dtype={'euros_per_mwh': 'float64',
                          'data_block_id': 'int64'},
                   parse_dates=['forecast_date', 'origin_date'])

historical_weather = pd.read_csv(os.path.join(PATH, 'historical_weather.csv'),
                dtype={'temperature': 'float64',
                        'dewpoint': 'float64',
                        'rain': 'float64',
                        'snowfall': 'float64',
                        'surface_pressure': 'float64',
                        'cloudcover_total': 'int16',
                        'cloudcover_low': 'int16',
                        'cloudcover_mid': 'int16',
                        'cloudcover_high': 'int16',
                        'winddirection_10m': 'int16',
                        'shortwave_radiation': 'float64',
                        'direct_solar_radiation' : 'float64',
                        'diffuse_radiation': 'float64',
                        'latitude': 'float64',
                        'longitude' : 'float64',
                        'data_block_id' : 'int64'},

                parse_dates=['datetime'])

forecast_weather = pd.read_csv(os.path.join(PATH, 'forecast_weather.csv'),
                dtype={'temperature': 'float64',
                        'dewpoint': 'float64',
                        'total_precipitation': 'float64',
                        'snowfall': 'float64',
                        'cloudcover_total': 'float64',
                        'cloudcover_low': 'float64',
                        'cloudcover_mid': 'float64',
                        'cloudcover_high': 'float64',
                        '10_metre_u_wind_component': 'float64',
                        '10_metre_v_wind_component': 'float64',
                        'direct_solar_radiation' : 'float64',
                        'surface_solar_radiation_downwards': 'float64',
                        'latitude': 'float64',
                        'longitude' : 'float64',
                        'data_block_id' : 'int64',
                        'hours_ahead': 'int16'},

                parse_dates=['origin_datetime', 'forecast_datetime'])

train = pd.read_csv(os.path.join(PATH, 'train.csv'),
                dtype={ 'county': 'int16',
                        'is_business': 'boolean',
                        'product_type': 'int8',
                        'target': 'float64',
                        'is_consumption': 'boolean',
                        'data_block_id' : 'int64',
                        'row_id' : 'int16',
                        'prediction_unit_id' : 'int16' },

                parse_dates=['datetime'])

client = pd.read_csv(os.path.join(PATH, 'client.csv'),
                dtype={ 'county': 'int16',
                        'is_business': 'boolean',
                        'product_type': 'int8',
                        'eic_count': 'float64',
                        'installed_capacity': 'float64',
                        'data_block_id' : 'int64'},

                parse_dates=['date'])

In [3]:
weather_station = pd.read_csv(os.path.join(PATH, 'weather_station_to_county_mapping.csv'),
                   dtype={'county_name': 'str',
                          'longitude': 'float64',
                          'latitude': 'float64',
                          'county': 'float64'})

weather_station.dropna(subset='county', inplace=True)
weather_station.drop(columns=['county_name'], inplace=True)
weather_station['county'] = weather_station['county'].astype('int')
weather_station[['latitude', 'longitude']] = weather_station[['latitude', 'longitude']].astype(float).round(1)

In [4]:
# Reduce forecast data

forecast_weather = forecast_weather.rename(columns = {'forecast_datetime': 'datetime'})
forecast_weather.drop(columns = 'origin_datetime', inplace=True)
forecast_weather['datetime'] = forecast_weather['datetime'].dt.tz_convert('Europe/Brussels').dt.tz_localize(None)

# Map to weather locations
forecast_weather[['latitude', 'longitude']] = forecast_weather[['latitude', 'longitude']].astype(float).round(1)
forecast_weather = forecast_weather.merge(weather_station, how='left', on=['latitude', 'longitude'])

# Some weather locations are outside any county
forecast_weather.dropna(subset='county', inplace=True)

forecast_weather['county'] = forecast_weather['county'].astype(int)

# Some county have many weather locations
forecast_weather = forecast_weather.groupby(by=['datetime', 'county', 'data_block_id']).mean().reset_index()
forecast_weather = SplitDateColumn(forecast_weather, column='datetime')

In [5]:
gas_prices.query('data_block_id == 1')

,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
0,2021-09-01,45.23,46.32,2021-08-31,1


In [6]:
train.query('data_block_id == 1')['datetime']

2928   2021-09-02 00:00:00
2929   2021-09-02 00:00:00
2930   2021-09-02 00:00:00
2931   2021-09-02 00:00:00
2932   2021-09-02 00:00:00
               ...        
5851   2021-09-02 23:00:00
5852   2021-09-02 23:00:00
5853   2021-09-02 23:00:00
5854   2021-09-02 23:00:00
5855   2021-09-02 23:00:00
Name: datetime, Length: 2928, dtype: datetime64[ns]

## <span style='color:#2563eb'>🔷 | <b></span>Scelenie zbiorów</b>

In [7]:
# train <- client
mergedData = train.merge(client.drop(columns=['date']), how='left', on = ['county', 'is_business', 'product_type', 'data_block_id'])

# train <- forecast weather
mergedData = mergedData.merge(forecast_weather, how='left', on=['datetime', 'county', 'data_block_id'])

# train <- energy prices
columns = ['euros_per_mwh', 'data_block_id']
epToMerge = electricity_prices[columns].copy()
epToMerge['datetime'] = electricity_prices['forecast_date'] + timedelta(days=1)
AddPrefixToColumns(epToMerge, ['euros_per_mwh'], 'elec_price_')
mergedData = mergedData.merge(epToMerge, how='left', on=['datetime', 'data_block_id'])

# train <- gas prices
columns = ['gas_highest_price_per_mwh', 'gas_lowest_price_per_mwh']
merge=['data_block_id']
gpToMerge = gas_prices.copy()
gpToMerge['datetime_date'] = gas_prices['forecast_date'] + timedelta(days=1)
AddPrefixToColumns(gpToMerge, ['highest_price_per_mwh', 'lowest_price_per_mwh'], 'gas_')
mergedData = mergedData.merge(gpToMerge[merge + columns], how='left', on=merge)

# train <- custom features
merge = ['county', 'is_business', 'product_type', 'data_block_id', 'is_consumption', 'datetime']

columns = ['target_week_ago']
trainMinus7 = train.copy()
trainMinus7['datetime'] = trainMinus7['datetime'] + timedelta(days=7)
trainMinus7['data_block_id'] = trainMinus7['data_block_id'] + 7
trainMinus7.rename(columns={'target' : 'target_week_ago'}, inplace=True)
mergedData = mergedData.merge(trainMinus7[merge + columns], how='left', on=merge)

columns = ['target_3_days_ago']
trainMinus3 = train.copy()
trainMinus3['datetime'] = trainMinus3['datetime'] + timedelta(days=3)
trainMinus3['data_block_id'] = trainMinus3['data_block_id'] + 3
trainMinus3.rename(columns={'target' : 'target_3_days_ago'}, inplace=True)
mergedData = mergedData.merge(trainMinus3[merge + columns], how='left', on=merge)

columns = ['target_14_days_ago']
trainMinus14 = train.copy()
trainMinus14['datetime'] = trainMinus14['datetime'] + timedelta(days=14)
trainMinus14['data_block_id'] = trainMinus14['data_block_id'] + 14
trainMinus14.rename(columns={'target' : 'target_14_days_ago'}, inplace=True)
mergedData = mergedData.merge(trainMinus14[merge + columns], how='left', on=merge)


mergedData.dropna(inplace=True)

In [8]:
DescribeData(mergedData)

Size: 1929268 x 40

                county  is_business  product_type  target  is_consumption  \
Number of Nans       0            0             0       0               0   

                datetime  data_block_id  row_id  prediction_unit_id  \
Number of Nans         0              0       0                   0   

                eic_count  ...  datetime_year  datetime_datetime  \
Number of Nans          0  ...              0                  0   

                datetime_time  datetime_date  elec_price_euros_per_mwh  \
Number of Nans              0              0                         0   

                gas_highest_price_per_mwh  gas_lowest_price_per_mwh  \
Number of Nans                          0                         0   

                target_week_ago  target_3_days_ago  target_14_days_ago  
Number of Nans                0                  0                   0  

[1 rows x 40 columns]
             county  product_type        target  \
count  1.929268e+06  1.929268e+06

## <span style='color:#2563eb'>🔷 | <b></span>Podział na zbiory do modelów</b>

In [9]:
mergedData.columns

Index(['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'data_block_id', 'row_id', 'prediction_unit_id',
       'eic_count', 'installed_capacity', 'latitude', 'longitude',
       'hours_ahead', 'temperature', 'dewpoint', 'cloudcover_high',
       'cloudcover_low', 'cloudcover_mid', 'cloudcover_total',
       '10_metre_u_wind_component', '10_metre_v_wind_component',
       'direct_solar_radiation', 'surface_solar_radiation_downwards',
       'snowfall', 'total_precipitation', 'datetime_minute', 'datetime_hour',
       'datetime_day', 'datetime_month', 'datetime_year', 'datetime_datetime',
       'datetime_time', 'datetime_date', 'elec_price_euros_per_mwh',
       'gas_highest_price_per_mwh', 'gas_lowest_price_per_mwh',
       'target_week_ago', 'target_3_days_ago', 'target_14_days_ago'],
      dtype='object')

In [10]:
features = [
    'is_business',
    'product_type',
    'is_consumption',
    'county',
    'temperature',
    'dewpoint',
    'cloudcover_high',
    'cloudcover_low',
    'cloudcover_mid',
    'cloudcover_total',
    '10_metre_u_wind_component',
    '10_metre_v_wind_component',
    'direct_solar_radiation',
    'surface_solar_radiation_downwards',
    'snowfall',
    'total_precipitation',
    'installed_capacity',
    'elec_price_euros_per_mwh',
    'datetime_hour',
    'gas_highest_price_per_mwh',
    'gas_lowest_price_per_mwh',
    'target_week_ago',
    'target_3_days_ago',
    'target_14_days_ago'
    ]
target_columns = ['target']

X = mergedData[features]
y = mergedData[target_columns]

# Splitting into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## <span style='color:#2563eb'>🔷 | <b></span>XGBoost</b>

In [11]:
clf = xgb.XGBRegressor (
    # Device ordinal, available options are cpu, cuda, and gpu.
    device = device, 
    enable_categorical=True,
    # Number of gradient boosted trees
    n_estimators = 1000,
    # Step size shrinkage used in update to prevents overfitting
    eta=0.1,
    # Activates early stopping. Validation metric needs to improve at least once in every early_stopping_rounds round(s) to continue training
    early_stopping_rounds=100,
    # L2 regularization term on weights. Increasing this value will make model more conservative
    reg_lambda = 1,
    # L1 regularization term on weights. Increasing this value will make model more conservative
    reg_alpha = 0,
    # Minimum loss reduction required to make a further partition on a leaf node of the tree
    gamma = 0,
 )

### ✨ <b>Uczenie</b>

In [12]:
clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)])

[0]	validation_0-rmse:833.75468	validation_1-rmse:847.28774
[1]	validation_0-rmse:756.22170	validation_1-rmse:768.57544
[2]	validation_0-rmse:686.96369	validation_1-rmse:698.40206
[3]	validation_0-rmse:625.05099	validation_1-rmse:635.56832
[4]	validation_0-rmse:569.92431	validation_1-rmse:579.67672
[5]	validation_0-rmse:520.63642	validation_1-rmse:529.57670
[6]	validation_0-rmse:476.95264	validation_1-rmse:485.27522
[7]	validation_0-rmse:438.08095	validation_1-rmse:445.86125
[8]	validation_0-rmse:403.90442	validation_1-rmse:411.29293
[9]	validation_0-rmse:373.55480	validation_1-rmse:380.68733
[10]	validation_0-rmse:346.75606	validation_1-rmse:353.59162
[11]	validation_0-rmse:323.29232	validation_1-rmse:329.92452
[12]	validation_0-rmse:302.82029	validation_1-rmse:309.37695
[13]	validation_0-rmse:285.05945	validation_1-rmse:291.47298
[14]	validation_0-rmse:269.35815	validation_1-rmse:275.65310
[15]	validation_0-rmse:255.69591	validation_1-rmse:262.04515
[16]	validation_0-rmse:243.89854	v

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=100,
             enable_categorical=True, eta=0.1, eval_metric=None,
             feature_types=None, gamma=0, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, ...)

### ✨ <b>Wyniki</b>

In [13]:
pd.DataFrame({'name': clf.feature_names_in_, 'importance': clf.feature_importances_}).sort_values(by='importance', ascending=False)

,name,importance
21,target_week_ago,0.747710
23,target_14_days_ago,0.122578
22,target_3_days_ago,0.022215
2,is_consumption,0.013186
12,direct_solar_radiation,0.010263
18,datetime_hour,0.010103
13,surface_solar_radiation_downwards,0.009423
14,snowfall,0.007916
16,installed_capacity,0.006114
15,total_precipitation,0.005953


## <span style='color:#2563eb'>🔷 | <b></span>Las losowy</b>

In [14]:
clf = RandomForestRegressor(max_depth=10, n_estimators=5)

### ✨ <b>Uczenie</b>

In [15]:
clf.fit(X_train, y_train)

/home/dany/.local/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=10, n_estimators=5)

### ✨ <b>Wyniki</b>

In [16]:
y_pred = clf.predict(X_test)

np.sqrt(MSE(y_test, y_pred)) 

163.84865528770513

## <span style='color:#2563eb'>🔷 | <b></span>Szukanie hiperparametrów</b>

In [17]:
model = xgb.XGBRegressor (
    # Device ordinal, available options are cpu, cuda, and gpu.
    device = device, 
    enable_categorical=True,
    # Number of gradient boosted trees
    n_estimators = 1000,
    # Step size shrinkage used in update to prevents overfitting
    eta=0.1,
    # Activates early stopping. Validation metric needs to improve at least once in every early_stopping_rounds round(s) to continue training
    early_stopping_rounds=100,
    # L2 regularization term on weights. Increasing this value will make model more conservative
    reg_lambda = 1,
    # L1 regularization term on weights. Increasing this value will make model more conservative
    reg_alpha = 0,
    # Minimum loss reduction required to make a further partition on a leaf node of the tree
    gamma = 0,
 )

# A parameter grid for XGBoost
params = {
        'n_estimators': [1000],
        'eta': [0.1, 0.3, 0.5],
        'reg_lambda': [0.5, 1, 2],
        }

In [18]:
clf = GridSearchCV(model, params, verbose=2)

clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=False)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


/home/dany/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [15:02:23] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[CV] END .........eta=0.1, n_estimators=1000, reg_lambda=0.5; total time=  22.4s
[CV] END .........eta=0.1, n_estimators=1000, reg_lambda=0.5; total time=  22.0s
[CV] END .........eta=0.1, n_estimators=1000, reg_lambda=0.5; total time=  22.0s
[CV] END .........eta=0.1, n_estimators=1000, reg_lambda=0.5; total time=  22.7s
[CV] END .........eta=0.1, n_estimators=1000, reg_lambda=0.5; total time=  23.2s
[CV] END ...........eta=0.1, n_estimators=1000, reg_lambda=1; total time=  23.9s
[CV] END ...........eta=0.1, n_estimators=1000, reg_lambda=1; total time=  23.0s
[CV] END ...........eta=0.1, n_estimators=1000, reg_lambda=1; total time=  23.4s
[CV] END ...........eta=0.1, n_estimators=1000, reg_lambda=1; total time=  22.7s
[CV] END ...........eta=0.1, n_estimators=1000, reg_lambda=1; total time=  22.8s
[CV] END ...........eta=0.1, n_estimators=1000, reg_lambda=2; total time=  22.4s
[CV] END ...........eta=0.1, n_estimators=1000, reg_lambda=2; total time=  22.4s
[CV] END ...........eta=0.1,

: 